# Binary Answer Type

This notebook demonstrates how to generate forecasting questions with binary (yes/no) answers. Binary questions are the simplest and most common type, perfect for event prediction, outcome forecasting, and decision-making scenarios.

In [1]:
%pip install -e ..
%pip install python-dotenv

from IPython.display import clear_output
clear_output()

## Set up the client

**Set your API key:**

- **Locally**: Create a `.env` file in this directory with `LIGHTNINGROD_API_KEY=your_key_here`
- **Google Colab**: Go to the Secrets section (🔑 icon in left sidebar) and add a secret named `LIGHTNINGROD_API_KEY`

In [2]:
from utils import get_secret
from lightningrod import LightningRod

api_key = get_secret("LIGHTNINGROD_API_KEY")
base_url = get_secret("LIGHTNINGROD_BASE_URL", "https://api.lightningrod.ai/api/public/v1")

lr = LightningRod(api_key=api_key, base_url=base_url)

## Binary Answer Type Overview

Binary questions have two possible answers: Yes/No or True/False. They're ideal for:
- Event prediction ("Will X happen?")
- Outcome forecasting ("Will Y succeed?")
- Decision-making scenarios ("Should we do Z?")

Binary questions work with any data source: News Search, Top Aggregated News, or Custom Documents.

In [3]:
from datetime import datetime, timedelta
from lightningrod import QuestionGenerator, WebSearchLabeler, QuestionRenderer, QuestionPipeline, AnswerType, AnswerTypeEnum, NewsSeedGenerator

answer_type = AnswerType(answer_type=AnswerTypeEnum.BINARY)

seed_generator = NewsSeedGenerator(
    start_date=datetime.now() - timedelta(days=30),
    end_date=datetime.now(),
    search_query="technology product launches",
)

question_generator = QuestionGenerator(
    instructions=(
        "Generate binary forecasting questions about technology product launches. "
        "Questions should be answerable with Yes or No."
    ),
    examples=[
        "Will Apple release a new iPhone this year?",
        "Will Google launch a new AI product in Q2?",
        "Will Microsoft release Windows 12 in 2025?",
    ],
    bad_examples=[
        "What features will the new iPhone have?",
        "When will Google launch the product?",
        "How much will Windows 12 cost?",
    ],
    answer_type=answer_type,
)

# Labeler automatically finds answers to questions using web search
labeler = WebSearchLabeler(
    answer_type=answer_type,
    confidence_threshold=0.5,
)

# Renderer formats the question output
renderer = QuestionRenderer(answer_type=answer_type)

pipeline_config = QuestionPipeline(
    seed_generator=seed_generator,
    question_generator=question_generator,
    labeler=labeler,
    renderer=renderer,
)

## Run the Pipeline

Generate binary questions from the news articles.

In [4]:
dataset = lr.transforms.run(pipeline_config, max_questions=10) # keep max questions low when testing

> Note: This can take a few minutes to complete processing.

## View Results

Inspect the generated questions and answers. Each sample contains `seed`, `question`, `label`, `prompt`, and optional `context` and `meta` fields. See [API.md](../API.md) for the complete sample structure.

In [5]:
%pip install pandas

from IPython.display import clear_output
clear_output()

In [6]:
import pandas as pd

# Download samples to memory
samples = dataset.download()
print(f"Generated {dataset.num_rows} samples\n")

# Convert cached samples to a list of dictionaries
rows = dataset.flattened()

df = pd.DataFrame(rows)
df

Generated 10 samples



,question.question_text,label.label,label.label_confidence,label.resolution_date,label.reasoning,label.answer_sources,prompt,seed.seed_text,seed.url,seed.seed_creation_date,seed.search_query,is_valid,meta.sample_id,meta.parent_sample_id,meta.processing_time_ms,meta.filter_reason
0,Will Atrium release the beta version of Zephyr...,1,1.0,2026-01-28T00:00:00,Multiple recent news articles confirm that Atr...,https://vertexaisearch.cloud.google.com/ground...,QUESTION:\nWill Atrium release the beta versio...,"Title: Atrium Launches AIR, Its New Innovation...",https://www.businesswire.com/news/home/2026012...,2026-01-26T00:00:00,technology product launches,True,8fcf9d94-31d7-4eb8-8311-5d3e94d1b9e8,e592a6a4-a89e-43e4-a845-32b38ac8a42d,6949.576,NaN
1,Will Acer release the additional Swift laptop ...,1,1.0,2026-01-13T00:00:00,Acer has confirmed that the Swift laptop model...,https://vertexaisearch.cloud.google.com/ground...,QUESTION:\nWill Acer release the additional Sw...,Title: CES laptop announcements put AI PC hard...,https://thetacomaledger.com/2026/01/26/ces-lap...,2026-01-26T00:00:00,technology product launches,True,7d1a5431-f94b-4e78-962f-4ddf47121857,b724d50e-6a39-4ff3-ab0f-3b488513a318,9279.244,NaN
2,Will Giga Computing and Syrma SGS Technology L...,1,0.9,2026-01-28T00:00:00,Giga Computing and Syrma SGS Technology Limite...,https://vertexaisearch.cloud.google.com/ground...,QUESTION:\nWill Giga Computing and Syrma SGS T...,Title: Giga Computing Launches Server Manufact...,https://www.tribuneindia.com/news/business/gig...,2026-01-28T00:00:00,technology product launches,True,6b1ec932-761e-49e5-a313-6c5aef4671c4,be63b1e0-6973-4d6d-a96c-94860c64f0ba,11359.393,NaN
3,Will Samsung launch an AI TV lineup with Googl...,1,1.0,2025-12-29T00:00:00,Samsung has officially announced its plans to ...,https://vertexaisearch.cloud.google.com/ground...,QUESTION:\nWill Samsung launch an AI TV lineup...,"Title: Notion launches in Arabic, marking a ma...",https://www.tahawultech.com/industry/technolog...,2026-01-26T00:00:00,technology product launches,True,ea4b0d86-768d-4f18-bf1f-efc702aec3fc,c62d9bbf-2314-4482-8c68-e3fff0980177,6110.177,NaN
4,Will CCSC Technology International Holdings Li...,Undetermined,0.8,NaN,CCSC Technology International Holdings Limited...,https://vertexaisearch.cloud.google.com/ground...,NaN,Title: CCSC Technology International Holdings ...,https://www.globenewswire.com/news-release/202...,2026-01-28T00:00:00,technology product launches,False,bd76250f-9405-479b-8758-876f1c2ca420,7582a161-b6c9-4d4a-9760-cfeafcaf6bb4,7298.291,Undetermined label
5,Will Apple release its first foldable iPhone b...,1,0.9,NaN,"Several prominent industry analysts, including...",https://vertexaisearch.cloud.google.com/ground...,QUESTION:\nWill Apple release its first foldab...,Title: Apple 2026 product launches: stunning m...,https://punemirror.com/technology/apple/apple-...,2026-01-27T00:00:00,technology product launches,True,033f8d2c-0118-4b23-95ca-ba82be460c38,f88f5309-9f58-4769-97a0-22b386eff5bf,9110.002,NaN
6,Will the Seattle Ultrasonics C-200 UltraSonic ...,1,1.0,2025-09-27T00:00:00,The Seattle Ultrasonics C-200 UltraSonic Chef'...,https://vertexaisearch.cloud.google.com/ground...,QUESTION:\nWill the Seattle Ultrasonics C-200 ...,Title: CES 2026: The weirdest tech we saw\n\nU...,https://mashable.com/article/ces-2026-weird-tech,2026-01-28T00:00:00,technology product launches,True,c1e2b42e-62e4-4d44-adac-308f06511d51,a40125db-d604-475d-829b-23f013257409,5426.919,NaN
7,Will CCSC Technology International Holdings Li...,Undetermined,0.9,NaN,CCSC Technology International Holdings Limited...,https://vertexaisearch.cloud.google.com/ground...,NaN,Title: CCSC Technology International Holdings ...,https://www.stocktitan.net/news/CCTG/ccsc-tech...,2026-01-28T00:00:00,technology product launches,False,dea96c3c-b940-4da5-912a-9a5b688fb428,87b2cbf5-e0a6-41c6-a317-a71bfb4d7f92,7460.586,Undetermined label
8,Will Unitree Robotics release a commercial rob...,Unde

## When to Use Binary vs Other Answer Types

**Use Binary when:**
- You need simple yes/no answers
- Predicting events or outcomes
- Building decision-making systems
- You want the simplest evaluation

**Consider other types when:**
- You need numeric values → Use Continuous
- You need multiple options → Use Multiple Choice
- You need open-ended explanations → Use Free Response